In [1]:
import pandas as pd
import os
import tensorflow as tf
import util
import numpy as np

In [2]:
################### 학습 관련 Parameters #####################
dim_embed = 256
dim_hidden = 256
dim_image = 4096
batch_size = 128

#learning_rate = 0.001
n_epochs = 1000

In [3]:
data_path = '/home1/dataset/Flickr30k'
feat_path = '/home1/im2txt/showandtell/feats.npy'
annotation_path = os.path.join(data_path, 'results_20130124.token')
print(annotation_path)
! cat $annotation_path | head

/home1/dataset/Flickr30k/results_20130124.token
1000092795.jpg#0	Two young guys with shaggy hair look at their hands while hanging out in the yard .
1000092795.jpg#1	Two young , White males are outside near many bushes .
1000092795.jpg#2	Two men in green shirts are standing in a yard .
1000092795.jpg#3	A man in a blue shirt standing in a garden .
1000092795.jpg#4	Two friends enjoy time spent together .
10002456.jpg#0	Several men in hard hats are operating a giant pulley system .
10002456.jpg#1	Workers look down from up above on a piece of equipment .
10002456.jpg#2	Two men working on a machine wearing hard hats .
10002456.jpg#3	Four men on top of a tall structure .
10002456.jpg#4	Three men on a large rig .
cat: write error: Broken pipe


In [4]:
annotations = pd.read_table(annotation_path, sep='\t', header=None, names=['image', 'caption'])
captions = annotations['caption'].values

In [5]:
print(captions)

[ 'Two young guys with shaggy hair look at their hands while hanging out in the yard .'
 'Two young , White males are outside near many bushes .'
 'Two men in green shirts are standing in a yard .' ...,
 'A man is leaning off of the side of a blue and white boat as it sits in a body of water .'
 'A man riding a small boat in a harbor , with fog and mountains in the background .'
 'A man on a moored blue and white boat with hills and mist in the background .']


In [6]:
feats = np.load(feat_path)

In [7]:
def preProBuildWordVocab(sentence_iterator, word_count_threshold=30): # borrowed this function from NeuralTalk
    #print('preprocessing word counts and creating vocab based on word count threshold %d' % (word_count_threshold, ))
    word_counts = {}
    nsents = 0
    for sent in sentence_iterator:
      nsents += 1
      for w in sent.lower().split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1
    vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
    print('filtered words from %d to %d' % (len(word_counts), len(vocab)))

    ixtoword = {}
    ixtoword[0] = '.'  # period at the end of the sentence. make first dimension be end token
    wordtoix = {}
    wordtoix['#START#'] = 0 # make first vector be the start token
    ix = 1
    for w in vocab:
      wordtoix[w] = ix
      ixtoword[ix] = w
      ix += 1

    word_counts['.'] = nsents
    bias_init_vector = np.array([1.0*word_counts[ixtoword[i]] for i in ixtoword])
    bias_init_vector /= np.sum(bias_init_vector) # normalize to frequencies
    bias_init_vector = np.log(bias_init_vector)
    bias_init_vector -= np.max(bias_init_vector) # shift to nice numeric range
    return wordtoix, ixtoword, bias_init_vector

wordtoix, ixtoword, bias_init_vector = preProBuildWordVocab(captions)

filtered words from 20326 to 2942


In [8]:
np.save('data/ixtoword', ixtoword)

In [9]:
! ls -al data

total 56
drwxrwxr-x 2 greenseer greenseer  4096 12월  8 22:41 .
drwxrwxr-x 5 greenseer greenseer  4096 12월 11 03:32 ..
-rw-rw-r-- 1 greenseer greenseer 46751 12월 11 12:28 ixtoword.npy


In [10]:
index = np.arange(len(feats))
np.random.shuffle(index)

feats = feats[index]
captions = captions[index]

In [11]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [12]:
! nvidia-smi

Sun Dec 11 12:28:55 2016       
+------------------------------------------------------+                       
| NVIDIA-SMI 352.63     Driver Version: 352.63         |                       
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX TIT...  Off  | 0000:01:00.0     Off |                  N/A |
| 22%   35C    P2    67W / 250W |   6279MiB / 12285MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX TIT...  Off  | 0000:02:00.0     Off |                  N/A |
| 22%   29C    P2    68W / 250W |   6280MiB / 12287MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [13]:
n_words = len(wordtoix)
print n_words

2943


In [14]:
maxlen = np.max( map(lambda x: len(x.split(' ')), captions) )
print maxlen

82


In [15]:
dim_image = np.int(dim_image)
dim_embed = np.int(dim_embed)
dim_hidden = np.int(dim_hidden)
batch_size = np.int(batch_size)
n_lstm_steps = np.int(maxlen+2)
n_words = np.int(n_words)
learning_rate = 0.001
print dim_image
print dim_embed
print dim_hidden
print batch_size
print n_lstm_steps
print n_words

4096
256
256
128
84
2943


In [16]:
with tf.device("/cpu:0"):
    Wemb = tf.Variable(tf.random_uniform([n_words, dim_embed], -0.1, 0.1), name='Wemb')

In [17]:
bemb = tf.Variable(tf.zeros([dim_embed]), name="bemb")

In [18]:
lstm = tf.nn.rnn_cell.BasicLSTMCell(dim_hidden, state_is_tuple=False)

In [25]:
encode_img_W = tf.Variable(tf.random_uniform([dim_image, dim_hidden], -0.1, 0.1), name='encode_img_W') #(4096, 256)

In [26]:
encode_img_b = tf.Variable(tf.zeros([dim_hidden]), name="encode_img_b")

In [27]:
embed_word_W = tf.Variable(tf.random_uniform([dim_hidden, n_words], -0.1, 0.1), name='embed_word_W') # (256, 2943)

In [28]:
embed_word_b = tf.Variable(tf.zeros([n_words]), name="embed_word_b")

In [29]:
image = tf.placeholder(tf.float32, [batch_size, dim_image])
sentence = tf.placeholder(tf.int32, [batch_size, n_lstm_steps])
mask = tf.placeholder(tf.float32, [batch_size, n_lstm_steps]) # sentence를 나중에 padding하는데, 이때 실제로 값이 있는 것만 체크

In [30]:
image_emb = tf.matmul(image, encode_img_W) + encode_img_b # (batch_size, dim_hidden)

In [31]:
print lstm.state_size

512


In [37]:
state = tf.zeros([batch_size, lstm.state_size])

In [38]:
loss = 0.0

In [39]:
print sentence[:, 0]
print tf.expand_dims(sentence[:, 0], 1)
print tf.range(0, batch_size, 1)
print tf.range(0, batch_size, 1)
print tf.expand_dims(tf.range(0, batch_size, 1), 1)
labels = tf.expand_dims(sentence[:, 0], 1) #뒤에 더미 차원하나 더 추가.
indices = tf.expand_dims(tf.range(0, batch_size, 1), 1)
concated = tf.concat(1, [indices, labels])
onehot_labels = tf.sparse_to_dense(concated, tf.pack([batch_size, n_words]), 1.0, 0.0) # (batch_size, n_words)
print labels
print indices
print concated
print onehot_labels
print tf.pack([batch_size, n_words])

Tensor("strided_slice_3:0", shape=(128,), dtype=int32)
Tensor("ExpandDims_4:0", shape=(128, 1), dtype=int32)
Tensor("range_4:0", shape=(128,), dtype=int32)
Tensor("range_5:0", shape=(128,), dtype=int32)
Tensor("ExpandDims_5:0", shape=(128, 1), dtype=int32)
Tensor("ExpandDims_6:0", shape=(128, 1), dtype=int32)
Tensor("ExpandDims_7:0", shape=(128, 1), dtype=int32)
Tensor("concat_1:0", shape=(128, 2), dtype=int32)
Tensor("SparseToDense_1:0", shape=(128, 2943), dtype=float32)
Tensor("pack_3:0", shape=(2,), dtype=int32)


In [40]:
mask

<tf.Tensor 'Placeholder_5:0' shape=(128, 84) dtype=float32>

In [42]:
with tf.variable_scope("RNN2"):
    for i in range(n_lstm_steps): # maxlen + 1
        if i == 0:
            current_emb = image_emb # shape=(128, 256)
        else:
            with tf.device("/cpu:0"):
                current_emb = tf.nn.embedding_lookup(Wemb, sentence[:,i-1]) + bemb

        if i > 0 : tf.get_variable_scope().reuse_variables()

        # state shape=(128, 256)
        output, state = lstm(current_emb, state) # (batch_size, dim_hidden)

        if i > 0: # 이미지 다음 바로 나오는건 #START# 임. 이건 무시.
            labels = tf.expand_dims(sentence[:, i], 1) #뒤에 더미 차원하나 더 추가. shape=(128, 1)
            indices = tf.expand_dims(tf.range(0, batch_size, 1), 1) # 배치 인덱스 shape=(128, 1)
            concated = tf.concat(1, [indices, labels]) # shape=(128, 2)
            onehot_labels = tf.sparse_to_dense(
                    concated, tf.pack([batch_size, n_words]), 1.0, 0.0) # (batch_size, n_words)

            logit_words = tf.matmul(output, embed_word_W) + embed_word_b # (batch_size, n_words)
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logit_words, onehot_labels)# shape=(128,)
            cross_entropy = cross_entropy * mask[:,i]#tf.expand_dims(mask, 1) # shape=(128,)

            current_loss = tf.reduce_sum(cross_entropy)
            loss = loss + current_loss

    loss = loss / tf.reduce_sum(mask[:,1:])

In [43]:
saver = tf.train.Saver(max_to_keep=50)

In [44]:
train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [45]:
tf.initialize_all_variables().run()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [46]:
start = 0
end = 128
current_feats = feats[start:end]
current_captions = captions[start:end]

In [47]:
current_captions

array([ 'Many sports people dressed in black swim wear , yellow cap and goggle boisterously swimming across a body of water with a surrounding plush vegetation .',
       'A man , standing with his back to the camera , films another man , facing him , with white ash trees in the background , working in a notebook .',
       'The man turns with the wave on his surfboard .',
       'A group of people working on computers while wearing headsets .',
       'An old woman wearing a sunhat and pink blouse examines a grassy , swampy field .',
       'Two children playing on a jungle gym .',
       'a band with two females playing instruments',
       'A boy pointing in a direction on a dirt road .',
       'A lady wearing a fancy costume and headdress while drinking from a can .',
       'A man is sleeping on the sidewalk near a busy street .',
       'A tanned man with dark glasses on , punk style hair and tattoos on arms and chest .',
       'An elderly woman with a bag walking under an airp

In [48]:
map(lambda cap: cap,current_captions)

['Many sports people dressed in black swim wear , yellow cap and goggle boisterously swimming across a body of water with a surrounding plush vegetation .',
 'A man , standing with his back to the camera , films another man , facing him , with white ash trees in the background , working in a notebook .',
 'The man turns with the wave on his surfboard .',
 'A group of people working on computers while wearing headsets .',
 'An old woman wearing a sunhat and pink blouse examines a grassy , swampy field .',
 'Two children playing on a jungle gym .',
 'a band with two females playing instruments',
 'A boy pointing in a direction on a dirt road .',
 'A lady wearing a fancy costume and headdress while drinking from a can .',
 'A man is sleeping on the sidewalk near a busy street .',
 'A tanned man with dark glasses on , punk style hair and tattoos on arms and chest .',
 'An elderly woman with a bag walking under an airport sign .',
 'A woman walking down a hallway next to beautiful scenery .

In [49]:
current_caption_ind = map(lambda cap: [wordtoix[word] for word in cap.lower().split(' ')[:-1] if word in wordtoix], current_captions)
current_caption_ind

[[302,
  2424,
  145,
  2199,
  1592,
  1740,
  1063,
  2329,
  1511,
  1,
  940,
  2469,
  1320,
  1467,
  1132,
  479,
  2547,
  2419,
  354,
  1132,
  2220,
  2851],
 [1132,
  1691,
  1511,
  597,
  354,
  2684,
  2000,
  382,
  859,
  1302,
  1511,
  810,
  1691,
  1511,
  1802,
  2688,
  1511,
  354,
  2462,
  1200,
  1592,
  859,
  1611,
  1511,
  216,
  1592,
  1132,
  626],
 [859, 1691, 913, 354, 859, 1281, 601, 2684, 2663],
 [1132, 1389, 2547, 145, 216, 601, 1915, 1470, 1551],
 [363, 142, 2585, 1551, 1132, 2469, 1679, 14, 1401, 1132, 947, 1511, 1836],
 [1869, 1844, 1888, 601, 1132, 2253, 1532],
 [1132, 789, 354, 1869, 27, 1888],
 [1132, 1665, 1098, 1592, 1132, 2504, 601, 1132, 1159, 1819],
 [1132, 1070, 1551, 1132, 2651, 1323, 2469, 779, 1470, 2773, 2011, 1132, 944],
 [1132, 1691, 1589, 1771, 601, 859, 2645, 1585, 1132, 84, 2221],
 [1132,
  1691,
  354,
  331,
  643,
  601,
  1511,
  2633,
  2179,
  2469,
  1858,
  601,
  1486,
  2469,
  521],
 [363, 1615, 2585, 354, 1132, 156

In [50]:
from keras.preprocessing import sequence
current_caption_matrix = sequence.pad_sequences(current_caption_ind, padding='post', maxlen=maxlen+1)
current_caption_matrix

Using TensorFlow backend.


array([[ 302, 2424,  145, ...,    0,    0,    0],
       [1132, 1691, 1511, ...,    0,    0,    0],
       [ 859, 1691,  913, ...,    0,    0,    0],
       ..., 
       [1579,  145, 2693, ...,    0,    0,    0],
       [1132, 1691,  354, ...,    0,    0,    0],
       [1132,  586, 1121, ...,    0,    0,    0]], dtype=int32)

In [51]:
current_caption_matrix \
= np.hstack( [np.full( (len(current_caption_matrix),1), 0), current_caption_matrix] ).astype(int)
current_caption_matrix

/home/greenseer/user/sweaterr/py27/local/lib/python2.7/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((128, 1), 0) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


array([[   0,  302, 2424, ...,    0,    0,    0],
       [   0, 1132, 1691, ...,    0,    0,    0],
       [   0,  859, 1691, ...,    0,    0,    0],
       ..., 
       [   0, 1579,  145, ...,    0,    0,    0],
       [   0, 1132, 1691, ...,    0,    0,    0],
       [   0, 1132,  586, ...,    0,    0,    0]])

In [52]:
current_mask_matrix = np.zeros((current_caption_matrix.shape[0], current_caption_matrix.shape[1]))
current_mask_matrix

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [53]:
nonzeros = np.array( map(lambda x: (x != 0).sum()+2, current_caption_matrix ))
nonzeros

array([24, 30, 11, 11, 15,  9,  8, 12, 15, 13, 17, 13, 12,  7, 14, 13, 29,
       13, 10,  8, 10, 11,  9,  9, 12, 10, 11, 13, 14, 24, 13,  7, 14, 15,
       10, 17, 32,  8, 24, 19, 12, 15, 13, 16, 16, 19, 11,  9,  7, 11, 17,
       13,  9, 15,  9, 11, 14, 24, 10, 12, 13, 13,  8, 10, 15, 10, 19, 12,
       15,  6, 11, 11, 10, 18,  8, 11,  8, 21, 12, 11,  8, 12, 17,  9, 10,
       13,  9, 15, 12, 10, 20, 11, 11,  8, 15, 11, 11, 10, 10, 10, 14, 17,
       12,  9,  8, 10, 16,  8, 11,  9, 23, 17,  9, 18, 10, 19, 12,  7,  8,
       19, 23, 14,  9, 15, 13, 23, 14, 10])

In [54]:
for ind, row in enumerate(current_mask_matrix):
    row[:nonzeros[ind]] = 1
current_mask_matrix

array([[ 1.,  1.,  1., ...,  0.,  0.,  0.],
       [ 1.,  1.,  1., ...,  0.,  0.,  0.],
       [ 1.,  1.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  1.,  1., ...,  0.,  0.,  0.],
       [ 1.,  1.,  1., ...,  0.,  0.,  0.],
       [ 1.,  1.,  1., ...,  0.,  0.,  0.]])

In [55]:
_, loss_value = sess.run([train_op, loss], feed_dict={
                image: current_feats,
                sentence : current_caption_matrix,
                mask : current_mask_matrix
                })

In [56]:
for epoch in range(n_epochs):
    #train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    for start, end in zip( \
            range(0, len(feats), batch_size),
            range(batch_size, len(feats), batch_size)
            ):

        current_feats = feats[start:end]
        current_captions = captions[start:end]

        current_caption_ind = map(lambda cap: [wordtoix[word] for word in cap.lower().split(' ')[:-1] if word in wordtoix], current_captions)

        current_caption_matrix = sequence.pad_sequences(current_caption_ind, padding='post', maxlen=maxlen+1)
        current_caption_matrix = np.hstack( [np.full( (len(current_caption_matrix),1), 0), current_caption_matrix] ).astype(int)

        current_mask_matrix = np.zeros((current_caption_matrix.shape[0], current_caption_matrix.shape[1]))
        nonzeros = np.array( map(lambda x: (x != 0).sum()+2, current_caption_matrix ))
        #  +2 -> #START# and '.'

        for ind, row in enumerate(current_mask_matrix):
            row[:nonzeros[ind]] = 1

        _, loss_value = sess.run([train_op, loss], feed_dict={
            image: current_feats,
            sentence : current_caption_matrix,
            mask : current_mask_matrix
            })

        print "Current Cost: ", loss_value

    print "Epoch ", epoch, " is done. Saving the model ... "
    saver.save(sess, os.path.join(model_path, 'model'), global_step=epoch)
    learning_rate *= 0.95

Current Cost:  7.87338
Current Cost:  7.81409
Current Cost:  7.65607
Current Cost:  7.45289
Current Cost:  7.16563
Current Cost:  6.92597
Current Cost:  6.59541
Current Cost:  6.33206
Current Cost:  6.15569
Current Cost:  5.9263
Current Cost:  5.8174
Current Cost:  5.75367
Current Cost:  5.66046
Current Cost:  5.47108
Current Cost:  5.43914
Current Cost:  5.49033
Current Cost:  5.49908
Current Cost:  5.3505
Current Cost:  5.43449
Current Cost:  5.39098
Current Cost:  5.37832
Current Cost:  5.31493
Current Cost:  5.37557
Current Cost:  5.34743
Current Cost:  5.381
Current Cost:  5.18915
Current Cost:  5.38126
Current Cost:  5.42363
Current Cost:  5.22728
Current Cost:  5.32384
Current Cost:  5.24502
Current Cost:  5.28615
Current Cost:  5.18055
Current Cost:  5.2543
Current Cost:  5.23243
Current Cost:  5.26076
Current Cost:  5.17178
Current Cost:  5.22548
Current Cost:  5.25364
Current Cost:  5.11297
Current Cost:  5.23065
Current Cost:  5.24449
Current Cost:  5.16836
Current Cost:  5.

NameError: name 'model_path' is not defined